##### 작업전(t일 이전) 할 거
- 미리 확인용 데이터 다운로드(하면 더 좋고, 안그럼 전산과 엑셀파일 분납메모 같이 수정하면 되고)
0. 엑셀파일 열어서
1. 항고사건 결과 확인 : (분납상태 : 진행) & (현재결과 : 항고) & (분납메모 : 미결정 or "") 인 분납건에 대해
    - 엑셀 L ~ AW(메모) ~ BR(현재결과) 그룹
    - 분납메모 양식 : 항고사건결과,항고사건결과일,항고사건확정일,확인일 (공백없이 입력)
2. 12월에 일반회생(연납건) 입금회차 확인
20420482 보증인
20420209 차주
20426827 차주
20426904 차주
입금월 12, 12, 12, 12

- 수정대상건에 대해 전산수정작업 마친 후 t+1일에 다운로드
1. 우클릭 후 '1.채권자목록,일반내용,진행내용 표시'선택할 것!!

##### import

In [1]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
from datetime import datetime as dt

# 파일 읽기
path_base = r"D:\3.자산\전산 dataset"
########################################################
company = "솔림"
workdate = "240528" # 폴더명
name_dataset = "개인회생새창_20240529_0950"
# previous_workdate = "240108" # 폴더명
# previous_name_dataset = "개인회생새창_20240109_0943_기준데이터" # 기준데이터 계좌키 미분리
########################################################
path_dataset = join(path_base, company, workdate)
# path_previous = join(path_base, company, previous_workdate, previous_name_dataset+".xlsx") # 전체시트 읽어야 하므로 pkl 아님
path_result = join(path_dataset, name_dataset+"_기준데이터.xlsx")

rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}

# 금지명령(중지명령) : 전산은 금지명령이 기각되는 경우를 무시하고 신청서제출만으로 금지로 처리함.
# 접수 : 금지명령신청만 한 경우나, 기각된 경우를 따로 표시하기 위해 추가함
항고인용사건_현재결과종류 = {"개시":"개인회생절차개시결정\x7f", "금지": r"금지명령\(|중지명령\x7f", # r"(금지|중지)명령신청서? 제출\x7f", 
          "면책허가결정":"면책허가결정공고\x7f", "신청취하" : "종국 : 신청취하\x7f", "이송" : "종국 : 이송\x7f", "인가" : r"(회생|변제)계획인가결정\x7f","인용" : "종국 : 인용\x7f",
          "폐지취소결정" :"폐지취소결정\x7f", "폐지취소결정송달" : "폐지취소결정 송달\x7f"}

기각폐지항고 = {"기각":"종국 : 기각\x7f", "기각결정":"개인회생절차개시신청 기각결정\x7f", 
          "폐지결정" : "폐지결정\(", "폐지":"종국 : 폐지\x7f", 
          "항고" : "항고장"}

현재결과종류 = {key: value for d in [항고인용사건_현재결과종류, 기각폐지항고] for key, value in d.items()}

def 현재결과찾기(진행내용:str) -> str:
    # 진행내용 최근 결정내용을 찾아 현재결과 리턴
    for value in reversed(진행내용.split("\n")) :
        for i, v in 현재결과종류.items() :
            if re.search(v, value) :
                return i
            
    return "접수" # 처음까지 왔는데 아무런 결정내용도 확인할 수 없다면

def 항고사건_현재결과찾기(진행내용:str) -> str :
    for value in reversed(진행내용.split("\n")) :
        for i, v in 항고인용사건_현재결과종류.items() :
            if re.search(v, value) :
                return i
    return "접수" # 처음까지 왔는데 아무런 결정내용도 확인할 수 없다면

In [2]:
# previous = pd.read_excel(path_previous, sheet_name="개인회생(전체)", dtype={'채무자키':str, '분납키':str, '계좌키':str})
path_raw_data = join(path_dataset, name_dataset+".xlsx")
raw_data_ori = pd.read_excel(path_raw_data, dtype=rehabilitation_dtype).fillna("")

if raw_data_ori.loc[0,"진행내용"] == "" :
    print("개인회생 엑셀다운로드시 우클릭후 '1.채권자목록,일반내용,진행내용 표시'를 선택해야 합니다.")
else : 
    pass

In [3]:
raw_data = raw_data_ori.copy()

##### 중복건 중 최신건만 남기기

In [4]:
# 접수일 없는 건은 모두 중단건이다.(자동으로 불러오지 못한 거니까). 따라서 일단 제일 뒤로 보내기 위해 임의의 날짜값으로 대체
raw_data.loc[raw_data.query('접수일==""').index,"접수일"] = "1975-01-01"
# 역정렬
raw_data = raw_data.sort_values(by='접수일', ascending=False)

# 삭제할 것 중 분납상태 진행 있는지 체크
삭제할중복값 = raw_data[raw_data.duplicated(subset=['계좌키', '분납자성명인'], keep='first')]
if len(삭제할중복값.query('분납상태 == "진행"')) : 
    display(삭제할중복값)
else : 
    print(len(raw_data))
    raw_data.drop_duplicates(subset=['계좌키', '분납자성명인'], keep='first', inplace=True)
    raw_data.reset_index(drop=True, inplace=True)
    print(len(raw_data))

9631
9235


##### 사전점검 : 동일 사건 중 현재결과 및 분납상태 다른 거 있는지 

In [5]:
동일사건 = raw_data.groupby(by=['관할법원', '사건번호'])
상이한현재결과 = 동일사건.filter(lambda x: x['현재결과'].nunique() != 1)
상이한현재결과.sort_values(["사건번호"])[["채무자키","분납키","주민번호인","분납자관계","분납자성명인","관할법원","사건번호","현재결과","분납상태"]]
# 분납상태가 모두 중단이면 현재결과 달라도 상관없음

,채무자키,분납키,주민번호인,분납자관계,분납자성명인,관할법원,사건번호,현재결과,분납상태
8216,20552818,20150218,401-81-45117,보증인,주종진,전주지방법원,2019개회10603,인용,중단
8217,20552818,20150217,401-81-45117,보증인,주종진,전주지방법원,2019개회10603,인용,중단
8219,20495098,20126689,401-81-45117,보증인,주종진,전주지방법원,2019개회10603,인가,중단
8220,20495130,20126688,401-81-45117,보증인,주종진,전주지방법원,2019개회10603,인가,중단


##### 인가미인가

In [6]:
# 컬럼 순서 바꾸기, 인트를 문자열로(타입때메 종료제거보다 나중에 나와야)
raw_data.insert(1, "분납키", raw_data.pop("분납키"))
raw_data.insert(7, "현재결과", raw_data.pop("현재결과"))
raw_data.insert(8, "분납상태", raw_data.pop("분납상태"))
raw_data.insert(9, "검수상태", raw_data.pop("검수상태"))

# 현재결과 종류
# 인가 : 인가
# 미인가 : 개시, 금지, 금지명령
# 폐지 : 절차폐지결정일, 폐지결정, 기각, 기각결정, 신청취하, 폐지
# 면책 : 인용, 면책허가결정, 면책,              * 종결은 면책목록에서 제외함
# (230131) 개편
# 변제계획인가일 확인 할 것 : 폐지취소결정송달, 폐지취소결정
# 분납메모, 변제계획인가일 확인 할 것

# 인가미인가
인가미인가 = []
for i, v in tqdm(raw_data[["현재결과", "분납키", "분납메모", "변제계획인가일", "항고", "진행내용"]].iterrows(), total=len(raw_data.index)) :
    result, opb = "", 0
    
    결과 = v.현재결과
    # 현재결과에 따라 인가,미인가,폐지,면책,이송,체크 정하기
    # 인가
    if 결과 == "인가" or (결과 =="종결" and re.search("일반회생", v.분납메모)): # 종결은 수기등록 하는 거 아니면 회단사건에서만 확인되고 있다. 
        result = "인가"                 # 분납메모가 있으면 회단사건의 종결은 인가와 다름없다.(법원 절차만 종결됐다는 소리이므로). 분납메모가 없다면 체크로 빠지도록 한다. 완제됐다면 분납상태에서 걸리겄지.
        
    # 미인가
    elif re.search("개시|금지|접수", 결과) : 
        result = "미인가"
        
    # 폐지
    elif (re.search("폐지결정|기각|취하", 결과)) or (결과 == "폐지") : 
        result = "폐지"
        
    # 폐지취소 - 분납메모 미사용
    elif re.search("폐지취소", 결과) : 
        result = "미인가" if v.변제계획인가일 == "" else "인가"
            
    # 면책
    elif re.search("인용|면책|완제", 결과) : # 일부 채권사것만 완제하는 경우가 있다. 종결로 적지 않고 완제로 적을 것!!!
        result = "면책"
        
    # 이송 : 분납상태 중지 & 새사건 입력되었어야
    elif re.search("이송", 결과) :
        result = "이송"
        
    # 실사필요 : 항고인데 현재결과가 어디에도 해당하지 않을 때 or 새로운 현재결과가 나온 경우
    elif re.search("항고", 결과) : 
        항고사건결과 = v.분납메모.split(",")[0]
        채무자항고 = False if re.search("채권", v.항고) else True
        
        # 항고사건결과 아직 없음 : 체크
        if 항고사건결과 == "" : 
            result = "체크_항고사건결과없음"
            
        # 미결정 확인 완료 : 인가일 있는지에 따라
        elif 항고사건결과 == "미결정" : 
            result = "미인가" if v.변제계획인가일 == "" else "인가"
            
        # 항고 인용, 기각취소, 폐지취소 ----------------ㅁㅁㅁ 현재 항고사건없이 바로 결정나온것에 기각취소, 기각, 폐지취소는 확인되었으나 폐지(?!취소)는 확인이 안 되어 코드에 미반영상태
        elif re.search("환송|인용|기각취소|폐지취소", 항고사건결과) :
            if 채무자항고 :
                result = "미인가" if v.변제계획인가일 == "" else "인가"
                raw_data.loc[i, "현재결과"] = 항고사건_현재결과찾기(v.진행내용)
            else : 
                result = "체크_채권자항고" # 채권자 항고에 대한 기각취소는 없어야하고, 파기환송과 인용은 아직 사례를 못 봐서 데이터 수집 필요
        # 취하, 기각, 각하
        elif re.search("취하|각하|기각(?!취소)", 항고사건결과) : # ----------------ㅁㅁㅁ 항고사건없이 항고장제출 후 폐지가 나오는 경우 있다면 추가해야
            if 채무자항고 : 
                result = "폐지"
            else : 
                result = "미인가" if v.변제계획인가일 == "" else "인가"
                raw_data.loc[i, "현재결과"] = 항고사건_현재결과찾기(v.진행내용)
        # 분납메모에 엉뚱한 말이 적힌 경우
        else : 
            result = "체크_항고사건결과를 잘못적음"
    
    # 위의 어떤 경우에도 해당하지 않는 경우 : 없어야 함
    else : result = "체크_항목에 없는 현재결과"

    인가미인가.append(result)

raw_data.insert(7, "인가미인가", 인가미인가)
raw_data.insert(11, "반영OPB", None)

# 폐지취소 이전 파일에서 불러왔던 코드
# elif re.search("폐지취소", 결과) :  # 폐지취소결정송달 포함
#     temp = previous.query('분납키==@v.분납키')["인가미인가"].values
#     if len(temp) == 1 : result = temp[0]
#     else : result = "체크"

100%|██████████| 9235/9235 [00:00<00:00, 46210.24it/s]


##### 체크

In [7]:
# 진행건 중 체크 건 있는지 확인
수작업 = raw_data.query("인가미인가.str.contains('체크') and 분납상태 =='진행'")[["채무자키","계좌키","분납키","주민번호인","인가미인가","현재결과","분납메모","항고일","변제계획인가일"]]
if len(수작업) > 0: 
    display(수작업)
    #수작업.to_excel("xxx.xlsx")
else : 
    print("진행건중 체크할 거 없음")

진행건중 체크할 거 없음


In [42]:
# # 확인 후 수정 or pass                               ####################
# raw_data.loc[[829, 830, 6205, 7176],["분납메모","인가미인가"]] = ["파기환송", "인가"]
raw_data.loc[[3339,7938],["분납메모","인가미인가"]] = ["미결정", "인가"]
                                                      ####################

In [8]:
# check ----------------------------
cols = ["채무자키", "분납키", "분납자관계", "분납자성명인", "인가미인가", "현재결과","현재결과일", "분납상태", "분납메모","중단사유", "분납중단일", "채무상태", "담당자", "개시결정일", "변제계획인가일", "사건번호"]
check = raw_data[cols].copy()
# 인가미인가 : 인가,미인가 // 폐지,면책,이송 // 체크
# 분납상태 : 진행, 중단

# 1. 인가미인가와 분납상태의 불일치 -> 인가 or 미인가 = 분납상태 진행
# 1-1.진행건 vs 상태 중단 : 중단사유 있는지?
중단사유적어 = check.query('인가미인가.str.contains("인가|미인가") & 분납상태=="중단" & 중단사유 == ""')
if len(중단사유적어) > 0 :
    print("중단사유 명시 필요") # 현재결과는 중단사유가 아니라 바로 확인 불가
    display(중단사유적어)
    
# 1-2. 중단건(인가/미인가 아닌것) vs 상태 진행
진행건_인가미인가오류 = check.query('분납상태=="진행" & ~인가미인가.str.contains("인가|미인가")')
if len(진행건_인가미인가오류) :
    print("진행건인데 인가나 미인가가 아닌 것 있음. 폐지 맞는지 확인(폐지결정 나더라도 바로 중단으로 하지 않는다. 2024-03~ / 인가미인가가 맞으면 됨.)")
    display(진행건_인가미인가오류)

# 2. not중지건 중 인가전 현재결과 vs 변제계획인가일 있는 경우
미인가건임 = check.query('분납상태=="진행" and 인가미인가=="미인가" and 변제계획인가일!=""')    ############## 20149032 는 회단사건으로 인가일 표기위치가 다름
인가건임 = check.query('분납상태=="진행" and 인가미인가=="인가" and 변제계획인가일=="" and 분납키!="20149032"')
if len(미인가건임) > 0 :
    print("미인가건인데 왜 인가일이 있음?")
    display(미인가건임)

if len(인가건임) > 0 :
    print("인가건인데 왜 인가일이 없음?, 일반회생과 종결건은 그럴 수 있음")
    display(인가건임)

# 3. not중지건 중 개시결정일 있는데 전단계인 경우
# (반대)왜인지는 몰라도 개시결정일을 못 읽는 경우가 1~2건 있네.. 
개시건임 = check.query('분납상태=="진행" and 개시결정일 != "" and 현재결과.str.contains("금지|접수")')
개시건인데 = check.query('분납상태=="진행" and 개시결정일 == "" and 현재결과.str.contains("개시") and 분납키!="20149032"')
if len(개시건임) > 0 :
    print("개시일 있는데 왜 전단계? - 항고건은 분납메모를 통해서 ")
    display(개시건임)
    
if len(개시건인데) > 0 :
    print("개시인데 왜 개시결정일 없음? 분납키 20145963, 20149575 개시임은 확인했는데 결정일을 못 읽네..")
    display(개시건인데)

# 4. not중지건 일반회생(회단,간회단) 중 분납메모 없는 건
미확인일반회생 = check.query('사건번호.str.contains("회단") and 분납상태=="진행" and 현재결과=="인가" and ~분납메모.str.contains("일반회생")')
if len(미확인일반회생) > 0 :
    print("인가된 일반회생인데 메모 왜 없니")
    display(미확인일반회생)

# 항고, 폐지취소 건 중 분납중지건은 확인 완료 (240130)
# 최신화 불가건 : 항고 뿐임 (폐지취소는 최신화 됨, 폐지 후 과거의 인가/미인가 여부를 확인하는 작업이 필요할뿐)
# 담당자, 채무상태, 누락 건 등은 자산정리 후 해야 정확

진행건인데 인가나 미인가가 아닌 것 있음. 폐지 맞는지 확인(폐지결정 나더라도 바로 중단으로 하지 않는다. 2024-03~ / 인가미인가가 맞으면 됨.)


,채무자키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,현재결과일,분납상태,분납메모,중단사유,분납중단일,채무상태,담당자,개시결정일,변제계획인가일,사건번호
344,20526268,20143971,채무자,천현경,폐지,폐지,2024-05-28,진행,,,,개인회생(폐지진행중),배정(대기),2023-08-31,2023-11-03,2023개회73606
794,20501293,20133470,채무자,정승봉,폐지,항고,2023-08-24,진행,기각,,,개인회생(폐지),배정(대기),2023-04-13,,2022개회14008
1143,20501640,20129753,채무자,윤현준,폐지,절차폐지결정일,2024-05-20,진행,,,,개인회생(폐지진행중),정현화,2022-07-26,2022-10-24,2022개회1054015
1583,20501823,20130304,채무자,최원철,폐지,절차폐지결정일,2024-05-22,진행,,,,개인회생(폐지진행중),정현화,2022-06-16,2022-11-18,2022개회1018996
1595,20501752,20130276,채무자,남선영,폐지,절차폐지결정일,2024-05-22,진행,,,,개인회생(폐지진행중),정현화,2022-04-27,2022-06-24,2022개회1019333
1745,20501617,20129804,채무자,김정남,폐지,폐지결정,2024-05-28,진행,,,,개인회생(폐지진행중),배정(대기),2022-05-25,2022-09-07,2022개회2217
1985,20501910,20130313,채무자,진종한,폐지,절차폐지결정일,2024-05-28,진행,,,,개인회생(폐지진행중),배정(대기),2022-05-11,2023-03-08,2021개회133395
2146,20501898,20130220,채무자,정미진,폐지,절차폐지결정일,2024-05-13,진행,,,,개인회생(폐지진행중),배정(대기),2022-06-10,2022-10-13,2021개회242279
2235,20501385,20130558,채무자,송호준,폐지,절차폐지결정일,2024-05-22,진행,,,,개인회생(폐지진행중),정현화,2022-04-11,2022-06-07,2021개회26829
2421,20501283,20130543,채무자,김한진,폐지,절차폐지결정일,2024-05-27,진행,,,,개인회생(폐지진행중),배정(대기),2022-03-16,2022-05-23,2021개회24427


인가건인데 왜 인가일이 없음?, 일반회생과 종결건은 그럴 수 있음


,채무자키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,현재결과일,분납상태,분납메모,중단사유,분납중단일,채무상태,담당자,개시결정일,변제계획인가일,사건번호
8762,20426904,20119739,채무자,임만수,인가,종결,2018-12-27,진행,"일반회생_3_1~9:41399,10:41395",,,개인회생(확정),개인회생(특이),,,2018회단100128
8763,20426827,20119733,채무자,임만수,인가,종결,2018-12-27,진행,"일반회생_3_1~9:41399,10:41400",,,개인회생(확정),개인회생(특이),,,2018회단100128
9144,20420482,20119684,보증인,이선숙,인가,종결,2019-02-22,진행,일반회생_5_1~10:2100000,,,파산(면책),신희종,,,2015회단502


In [10]:
# ### 체크 후 수정
# raw_data.loc[[14],["개시결정일","인가미인가"]] = ["파기환송", "미인가"]
# raw_data.loc[14,"개시결정일"] = '2024-04-02'

In [9]:
# 일반내용 ~ 필터제출기준현재회차 내용삭제
ind1 = raw_data.columns.get_loc('일반내용')
ind2 = raw_data.columns.get_loc('필터제출기준현재회차')
raw_data.iloc[:,ind1:ind2] = ""

In [10]:
# 금액 체크
# 입금이 잘못된 경우 초과입금이 생기는데, 이를 더해주지 않고 있으므로 - 값이 나올 수 있다.
# 초과입금은 회차별 변제금액 이외의 금액이 있는 경우 발생한다.(ex-3만원 변제인데 4만원 입금들어온경우)
display(raw_data.query('분납상태=="진행" and 초과입금>0'))
# 총분납금 < 총분납입금 : 말 그대로의 초과입금 (위에 포함됨)
display(raw_data.query('분납상태=="진행" and 총분납금<총분납입금'))

,채무자키,분납키,채권구분,매각사구분,분납자관계,분납자성명인,계좌키,인가미인가,현재결과,분납상태,...,중지명령일,사건키,자동최신화,최신화결과화면,관련최신화결과화면,공고게시일,공고물,공고변제율,공고변제기간,NOSER
409,20412588,20143704,NPL,KB카드-01,채무자,이창수,200973657,인가,인가,진행,...,,20546318,D,Y,,,,0,0,
1014,20408793,20137881,IRL,대산,채무자,김영용,200945591,인가,인가,진행,...,,20512180,D,Y,,,,0,0,
2058,20526112,20137628,IRL(인가),카카오-01,채무자,김아름,201059777,인가,인가,진행,...,,20511931,D,Y,,,,0,0,
2216,20501060,20130393,IRL(인가),스마트저축-01,채무자,박소이,201028738,인가,인가,진행,...,,20493852,D,Y,,,,0,0,
2365,20501804,20130307,IRL(미인가),스마트저축-01,채무자,김한빛,201029482,인가,인가,진행,...,,20493766,D,Y,,,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8542,20426620,20109070,IRL,대산,채무자,전동일,200945391,인가,인가,진행,...,,20397500,D,Y,,,,0,0,
8684,20408718,20104337,IRL,대산,채무자,이현,200945352,인가,인가,진행,...,,20377591,D,Y,,,,0,0,
8708,20410941,20104470,IRL,아인스,채무자,김영미,200976294,인가,폐지취소결정송달,진행,...,,20377724,D,Y,,,,0,0,
9065,20408896,20104140,IRL,대산,보증인,신미선,200945836,인가,인가,진행,...,,20377394,D,Y,,,,0,0,


,채무자키,분납키,채권구분,매각사구분,분납자관계,분납자성명인,계좌키,인가미인가,현재결과,분납상태,...,중지명령일,사건키,자동최신화,최신화결과화면,관련최신화결과화면,공고게시일,공고물,공고변제율,공고변제기간,NOSER
1014,20408793,20137881,IRL,대산,채무자,김영용,200945591,인가,인가,진행,...,,20512180,D,Y,,,,0,0,
2058,20526112,20137628,IRL(인가),카카오-01,채무자,김아름,201059777,인가,인가,진행,...,,20511931,D,Y,,,,0,0,
2641,20408051,20125172,NPL,한울가람,보증인,김만진,200922034,인가,인가,진행,...,,20449262,D,Y,,,,0,0,
2900,20527680,20136635,IRL(인가),카카오-01,채무자,소준,201061345,인가,인가,진행,...,,20510938,D,Y,,,,0,0,
2901,20526543,20136625,IRL(인가),카카오-01,채무자,소준,201060208,인가,인가,진행,...,,20510928,D,Y,,,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8542,20426620,20109070,IRL,대산,채무자,전동일,200945391,인가,인가,진행,...,,20397500,D,Y,,,,0,0,
8684,20408718,20104337,IRL,대산,채무자,이현,200945352,인가,인가,진행,...,,20377591,D,Y,,,,0,0,
8708,20410941,20104470,IRL,아인스,채무자,김영미,200976294,인가,폐지취소결정송달,진행,...,,20377724,D,Y,,,,0,0,
9065,20408896,20104140,IRL,대산,보증인,신미선,200945836,인가,인가,진행,...,,20377394,D,Y,,,,0,0,


##### 특이건 회차,금액 정리

###### 통합신고,제외

In [11]:
# 통합신고, 제외
# 통합신고는 애초에 하나의 계좌로 금액을 합산하여 신고한 경우로 신고채권번호도 한 개. 
#       단일채무자키인 경우 전산 통합등록하고, 아무계좌에서나 변제처리 되어도 상관없다. - 통합등록한 경우 계좌 분리과정에서 대표계좌만 금액이 남는다.
#       다채무자키인 경우 통합등록은 불가, 따라서 대표가 아닌 계좌의 분납메모에는 반드시 통합신고 적어줘야 금액 0원처리 가능
# 제외는 계좌별로 신고하였으나, 원금이 적어 총분납금이 0원인 채권으로 신고채권번호는 여러개이나, 법원채권번호는 하나만 나온 경우
#       신규건은 분리등록이 원칙. 이미 통합등록이 된 경우 분리등록해줘야 하나 그대로 두고 제외가 아닌 통합신고로 처리해도 별 문제는 없음
####### 통합신고채권(채권번호 하나)은 계좌별로 등록할 필요가 없으니 이렇게 적을 필요가 없음(아래서, 계좌통합건은 행분열하고 모두 0원으로 처리, 현재원금은 유지)
####### 통합신고채권은 계좌별로 분리하지 말고 통합해서 전산등록해야
#240131 정정
raw_data.loc[raw_data.분납메모.str.contains("통합|공채권|제외"), ["총분납금","총분납입금", "분납미납금", "분납잔금", "총분납잔금"]] = 0 # 입금내역 있는 것 재입금처리 해야함..

# 이전 코드
# raw_data.loc[raw_data.분납메모.str.contains("통합|공채권|제외"), ["총분납금","분납미납금"]] = 0
# raw_data.loc[raw_data.분납메모.str.contains("통합|공채권|제외"), ["분납잔금","총분납잔금"]] = raw_data['총분납금'] - raw_data["총분납입금"]

# # 엑셀 파일로 저장
# if not os.path.exists(path_result):
#     with pd.ExcelWriter(path_result, mode='w', engine='openpyxl') as writer:
#         raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)
# else : 
#     with pd.ExcelWriter(path_result, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer :
#         raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

In [12]:
# 공채권류 확인
raw_data.loc[raw_data.분납메모.str.contains("(공채권|통합|제외)"),["채무자키","분납키","주민번호인", "분납메모", "총분납금","총분납입금", "분납미납금", "분납잔금", "총분납잔금"]] #.to_excel(join(path_dataset, "공채권.xlsx"))

,채무자키,분납키,주민번호인,분납메모,총분납금,총분납입금,분납미납금,분납잔금,총분납잔금
476,20452190,20144370,970516-1249617,제외,0,0,0,0,0
655,20452468,20140956,750129-2006021,제외,0,0,0,0,0
2012,20451815,20125237,810710-1777411,제외,0,0,0,0,0
3193,20436703,20125480,571110-2449218,제외,0,0,0,0,0
3640,20421340,20146420,680603-1382111,통합신고 200932731,0,0,0,0,0
5175,20452192,20119998,750208-2468511,제외,0,0,0,0,0
5416,20452298,20146418,910706-2177813,제외,0,0,0,0,0
5423,20451911,20119946,930204-1320930,제외,0,0,0,0,0
5742,20452458,20120039,911231-2235625,제외,0,0,0,0,0
5959,20410192,20105691,800227-1726215,제외,0,0,0,0,0


In [41]:
######## 예전 수작업 내용 (지금은 모두 자동화했으므로 pass)
# 전산조회 할 거 작업 후
# - 분납상태-[진행] & 
# - 현재결과-[폐지취소결정, 폐지취소결정송달, 빈셀] & 
# - 인가미인가-[기타, #N/A, ""] 즉 인가,미인가가 아닌 거
# - 매각 제외 전이므로, 채무상태 및 담당자에서 종결, 매각건 제외

###### 연납건
- 분납메모가 있어야만 작성할 수 있다.

In [13]:
# 연납건 중 인가건 처리-----------------------------------------------------------------------------------------------------------------------------------
# 나우리의 계산법과 다르게 컷오프전 입금회차를 포함함.(즉 언제나 1회차 부터계산)
연납건index = raw_data.query('분납메모.str.contains("일반회생") and 분납상태=="진행"').index

if re.search("\d{6,6}",workdate) :
    real_workdate = workdate
else : real_workdate = dt.strftime(dt.today(), "%y%m%d")

for i in 연납건index : 

    기본정보 = raw_data.loc[i,"분납메모"].split("_")
    시작일 = dt.strptime(raw_data.loc[i,"분납시작일"], "%Y-%m-%d")
    종료일 = dt.strptime(raw_data.loc[i,"분납종료일"], "%Y-%m-%d")

    # 회차
    총분납회차 = 종료일.year - 시작일.year + 1
    if real_workdate[2:] < str(시작일.month)+str(시작일.day) : 
        현재회차 = int("20"+real_workdate[:2]) - 시작일.year # ex) 2022년 9월 30일, 분납시작일 2021년 12월 1일 > 1회차
    else : 
        현재회차 = int("20"+real_workdate[:2]) - 시작일.year + 1 # ex) 2022년 12월 30일, 분납시작일 2021년 12월 1일 > 2회차
    납입회차 = int(기본정보[1]) + raw_data.loc[i,"매입전변제금"] # 솔림입금회차 + 매입전 입금회차
    잔여회차 = 총분납회차 - 현재회차
    미납회차 = 현재회차 - 납입회차

    # 금액
    _회차별_변제금_ = 기본정보[-1].split(",")

    회차별_변제금 = pd.Series(index=range(1,총분납회차+1), name="회차별변제금")
    for v in _회차별_변제금_ :
        회차구간, 회차구간별변제금 = v.split(":")
        회차구간별변제금 = int(회차구간별변제금)

        if re.search("~", 회차구간) : 
            sn = int(회차구간.split("~")[0])
            en = int(회차구간.split("~")[1])
            회차별_변제금.loc[sn:en] = 회차구간별변제금
        else : 
            회차별_변제금.loc[int(회차구간)] = 회차구간별변제금

    총분납금 = 회차별_변제금.sum()
    총분납입금 = 회차별_변제금.loc[:납입회차].sum()
    총분납잔금 = 총분납금 - 총분납입금 # = 분납잔금 + 분납미납금 
    분납잔금 = 회차별_변제금.loc[현재회차+1 : ].sum() # 미래
    분납미납금 = 총분납금 - 총분납입금 - 분납잔금 # 과거 # 구간별 변제금이 다를 수 있으므로 미납회차*변제금을 할 수 없음

    raw_data.loc[i, "총분납회차"], raw_data.loc[i, "현재회차"], raw_data.loc[i, "납입회차"], raw_data.loc[i, "잔여회차"], raw_data.loc[i, "미납회차"] = 총분납회차, 현재회차, 납입회차, 잔여회차, 미납회차
    raw_data.loc[i, "총분납금"],raw_data.loc[i, "총분납입금"], raw_data.loc[i, "분납미납금"], raw_data.loc[i, "분납잔금"], raw_data.loc[i, "총분납잔금"] = 총분납금, 총분납입금, 분납미납금, 분납잔금, 총분납잔금


##### 반영OPB 및 요약

In [14]:
# OPB : 현재법비용 추가일 2024-04-03
raw_data['반영OPB'] = np.where(raw_data['인가미인가'] == '인가', raw_data["총분납잔금"] , raw_data["현재원금"] + raw_data["현재법비용"])


In [15]:
전체요약 = raw_data.query('분납상태=="진행"').groupby(['인가미인가','현재결과']).agg({'분납키':'count', '반영OPB':'sum'})
전체요약.loc[("합계", ""), :] = 전체요약.sum()
전체요약.applymap('{:,.0f}'.format)

분납키           반영OPB
인가미인가 현재결과                           
면책    면책허가결정        1       8,317,958
미인가   개시          152   1,277,480,556
      금지          128   1,292,109,482
      폐지취소결정송달      2      31,362,790
      항고            4      18,560,969
인가    인가        4,079   7,219,229,243
      종결            3       4,696,785
      폐지취소결정        5       5,436,586
      폐지취소결정송달     74     109,680,242
      항고            5      11,756,700
폐지    절차폐지결정일      25     219,627,408
      폐지            1       6,782,380
      폐지결정          3       9,531,300
      항고            2      17,341,457
합계              4,484  10,231,913,856

##### 진행건 분리 - 종결건 제외하지 않아야 함

In [16]:
# 진행중, 차주, 보증인 시트 작성(종결인데 분납중지 안 된 것 여기선 제외x)
ing_df = raw_data.query('분납상태 == "진행" and 인가미인가.str.contains("인가|미인가")')

# 종료건 제외 : 일부 계좌만 종결|매각|환매 될 수 있으므로 여기서 제외하지 않는다. -> 종료건 제외도 계좌별 상태가 확정되고 해야 함
# 어차피 통합에선 종결건은 우선 제외하므로 자산정리에 문제는 없음
# 통합파일에서 계좌별로 상태 확인 후 종료인데 중지 안 된 건 처리
# 최초 작성코드
# if path_dataset.find("대성") < 0 : # 솔림
#     ing_df = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각(?!예정)|DS|환매"))]
# else : # 대성
#     ing_df = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각|환매|파산|회계파산"))]

    
###### 신복과 충돌하는 거 제거하기 #############################################################
if path_dataset.find("대성") < 0 : # 솔림
    개회제외 = {"채무자" : [], "보증인" : [] } # 20462830 중복입금건. 신복으로 처리하다가 개인회생 면책됨 231106
else : # 대성
    개회제외 = {"채무자" : [], "보증인" : [] } # 
##############################################################################################
if len(개회제외["채무자"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["채무자"]) & (ing_df['분납자관계'] == '채무자'))]
if len(개회제외["보증인"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["보증인"]) & (ing_df['분납자관계'] == '보증인'))]

# 채무자, 보증인
debt_ing = ing_df[ing_df["분납자관계"]=="채무자"]
grt_ing = ing_df[ing_df["분납자관계"]=="보증인"]

# 저장
with pd.ExcelWriter(join(path_dataset, name_dataset+"_기준데이터.xlsx"), engine='openpyxl', mode = 'w') as writer :
    ing_df.to_excel(writer, sheet_name='개인회생(진행)', index=False)
    debt_ing.to_excel(writer, sheet_name='개인회생(진행)_채무자', index=False)
    grt_ing.to_excel(writer, sheet_name='개인회생(진행)_보증인', index=False)
    raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

##### 계좌키 나누고 새로운 파일 저장

In [17]:
# 개인회생 계좌 나누기

# path_dataset = r"D:\3.자산\전산 dataset\230430(0501)"
# name_dataset = "개인회생새창_20230502_0850_기준데이터"
# ing_df = pd.read_excel(join(path_dataset, name_dataset+".xlsx"), sheet_name="개인회생(진행)", dtype={'채무자키':str, '분납키':str, '계좌키':str})

if len(ing_df.계좌키.apply(len).value_counts()) > 1 : # 계좌키의 길이값이 두 종류 이상이라면(통합계좌 있으면)

    split_ing_df = ing_df.copy()
    split_ing_df["비고"] = ""
    new_rows = []
    for index, row in split_ing_df.iterrows():
        account_keys = row['계좌키'].split(',')
        first_account_key = account_keys.pop(0)
        split_ing_df.loc[index, '계좌키'] = first_account_key # 여러 계좌키를 첫번째 계좌키로 값 수정

        for account_key in account_keys:
            new_row = row.copy() # 일단 모든 칼럼 복사
            new_row['계좌키'] = account_key
                                # 전산 OPB는 최초OPB임
            new_row['반영OPB'],new_row['OPB'], new_row['총분납금'],new_row['총분납입금'],new_row['분납미납금'],new_row['분납잔금'],new_row['총분납잔금'] = 0,0,0,0,0,0,0 # 값 안 가져올 컬림은 빈문자열로 대체
            new_row["분납메모"] = "통합신고 " + first_account_key
            new_rows.append(new_row)
            

    temp = pd.DataFrame(new_rows)

    result = pd.concat([split_ing_df, temp], ignore_index=True)

    # 저장
    writer = pd.ExcelWriter(join(path_dataset, name_dataset+"_진행중_계좌키분리_기준데이터.xlsx"))
    result.to_excel(writer, sheet_name="개인회생(진행)", index=False)
    # writer.save() # 이제 save()는 없어진 api 안 쓰면 됨.
    writer.close()


In [18]:
# 면책건

# cols = ["채무자키", "계좌키", "분납자관계", "분납자성명인", "인가미인가", "현재결과", "분납상태", "분납중단일", "중단사유","현재결과일","면책결정일"]
면책 = raw_data.query('인가미인가=="면책" or 중단사유 in ["면책","납부완료","중도완제"] or 면책결정일!=""').copy()

# 계좌키를 쉼표로 분리하여 여러 개의 행으로 분할
splitted_df = pd.DataFrame(면책['계좌키'].str.split(',').tolist(), index=면책.index).stack() # 스택함수를 쓰면 열이 행으로 분리되고, level_0(원래 인덱스), level_1 의 이차원 인덱스가 생긴다.

# 인덱스 재설정
splitted_df = splitted_df.reset_index(level=1, drop=True).rename('분리계좌키') # 레벨0만 남기고 1은 드랍

# 합치기
면책_행분리 = 면책.merge(splitted_df, how='left', left_index=True, right_index=True)

면책_행분리.drop(['계좌키'],axis=1, inplace=True)
면책_행분리.insert(6,"분리계좌키", 면책_행분리.pop("분리계좌키"))
면책_행분리.rename(columns={"분리계좌키" : "계좌키"}, inplace=True)

면책_행분리.to_excel(join(path_dataset, name_dataset+"_면책건_계좌키분리.xlsx"), index=False)

### [DC] CHECK

In [19]:
# 보증인새창조회
####################################################
wd = r"D:\3.자산\전산 dataset\솔림\240528"
grt_ori = pd.read_excel(join(wd, "보증인새창_20240528_1729.xlsx"), dtype=grt_dtype).fillna("")
####################################################

#### 개인회생과 차주상태

In [20]:
# 면책건
차주면책건 = 면책_행분리.query('분납자관계=="채무자"')
보증인면책건 = 면책_행분리.query('분납자관계=="보증인"')

# 면책종결 관련 단어
면책종결 = r"종결|종료|면책|면탈|완납|완제|매각(?!예정)|(?<!피)환매|DS"

# 조회할 칼럼
cols = ["채무자키", "계좌키", "분납키", "분납자관계", "분납자성명인", "인가미인가","현재결과","분납중단일","중단사유","면책결정일","채무상태","담당자","주민번호인"]

In [21]:
# 차주상태 불일치건(기준일 면책건), 누락채권은 누락이 맞음. 
# 동일주민번호에 따른 채무상태 일치여부는 따로 확인하므로 개인회생데이터만봐도 됨.
check차주면책 = 차주면책건.query('~(채무상태.str.contains(@면책종결) or 담당자.str.contains(@면책종결))')[cols]
if len(check차주면책) > 0 :
    print("check차주면책")
    check차주면책.to_excel(join(wd,"[check]최근면책건_차주.xlsx"), index=False)
else :
    display(check차주면책)
# 누락건 제외하고는 계좌조회새창의 채무상태를 개인회생(면책)으로.

#### 개인회생과 보증인상태

In [22]:
grt = grt_ori.copy()

In [23]:
# 상태 '종결' 제외
grt1 = grt.query('~보증인상태.str.match("종결|보증면탈")')

# 언더바 제거
grt1["보증인상태"] = grt1.보증인상태.str.replace("_", "")

# 일단 보증인 상태값에 불일치 값이 없는지 체크
보증상태값중복 = grt1.groupby(by=["주민번호인"]).filter(lambda x: x.보증인상태.nunique() != 1)[["채무자키","채무자주민번호", "계좌키", "보증인키", "보증인성명인", "주민번호인", "보증인상태"]].sort_values(by=["주민번호인"])
보증상태값중복.to_excel(join(wd, "[check]보증상태값중복.xlsx"), index=False)

In [24]:
# 보증인상태 불일치건

# 개인회생새창에는 보증인 주민번호가 없으므로 보증인 주민번호를 먼저 찾아야함.
merged1 = 보증인면책건[cols].merge(grt[["채무자키", "보증인성명인", "주민번호인"]], left_on=["채무자키", "분납자성명인"], right_on=["채무자키", "보증인성명인"], how='left', \
    suffixes=("","_보증인")).drop(["보증인성명인"], axis=1)

# 인가미인가와 보증인 상태를 비교
merged2 = merged1.merge(grt[["주민번호인", "보증인상태"]], left_on=["주민번호인_보증인"], right_on=["주민번호인"], suffixes=("_채무자","_y"), how='left').drop(["주민번호인_y"], axis=1)

# 보증인 개회면책인데 보증인상태가 종결류가 아닌 것
merged3 = merged2.query('~보증인상태.str.contains("종결|면책|면탈|완납", na=False) and ~담당자.str.contains("종결")')

if len(merged3) > 0 :
    print("[check]보증인상태면책수정")
    merged3.to_excel(join(wd, "[check]보증인상태면책수정.xlsx"), index=False)
else : 
    display(merged3)

[check]보증인상태면책수정


In [25]:
merged2.query('주민번호인_보증인.isin(@보증상태값중복.주민번호인)')

,채무자키,계좌키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,분납중단일,중단사유,면책결정일,채무상태,담당자,주민번호인_채무자,주민번호인_보증인,보증인상태
